In [1]:
import findspark
import pyspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.master('spark://172.27.80.1:7077').appName('missing').getOrCreate()

In [6]:
df = spark.read.csv('file:///C:/pythonworks/210714/ContainsNull.csv', inferSchema=True, header=True)

In [7]:
df.head(2)

[Row(Id='emp1', Name='John', Sales=None),
 Row(Id='emp2', Name=None, Sales=None)]

In [15]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



### 이상치 삭제

- df.na.drop(how='any', thresh=None, subset=None)
- param how : 'any'(Null값 다 지우기), 'all'(Null, None 다 처리)

In [10]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [11]:
#몇개까지 허가할거냐
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [ ]:
df.na.drop(subset)

In [12]:
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [13]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [14]:
#빈값도 채우려면 도메인 전문가에게 물어봐라 -> 빈칸은 5로 채워주세요 등
#다른 타입 동시 처리는 어렵다. 그럴 일도 거의 없고

df.na.fill('NEW VALUE').show() #string만 처리됨 - new value가 str이니까

+----+---------+-----+
|  Id|     Name|Sales|
+----+---------+-----+
|emp1|     John| null|
|emp2|NEW VALUE| null|
|emp3|NEW VALUE|345.0|
|emp4|    Cindy|456.0|
+----+---------+-----+



In [16]:
df.na.fill(0).show() #int만 처리됨 - 0이 int이니까

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [17]:
#Name에 대해서 처리
df.na.fill('No Name', subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [18]:
#show = 보여줄 수 있는 데이터만 가져오기 actoin함수
#collect = 실제 작업이 이루어짐(드라이버 메모리 부족하면 죽어버릴 수도) transfrom 함수?

from pyspark.sql.functions import mean
mean_val = df.select(mean(df['Sales'])).collect() #collect는 실제 드라이버로 가져옮

In [24]:
#value x / row타입 객체
mean_val[0]

Row(avg(Sales)=400.5)

In [25]:
#이게 값
mean_val[0][0]

400.5

In [26]:
mean_sales = mean_val[0][0]

[sales col의 null값을 평균으로 채우기]

In [27]:
df.na.fill(mean_sales, ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [29]:
#위와 동일
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0], ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

